In [ ]:
import pandas as pd, numpy as np
from rolling_framework import Machine

import warnings
warnings.filterwarnings('ignore')

/Users/ethan_hong/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Loading

In [ ]:
period = '19712023'

#%% Target variables : excess returns
y = pd.read_csv(f'data/{period}/exrets_{period}.csv', index_col='Time')             # Target : Excess Return (Bond Risk Premia)
y = y[['xr_2','xr_5','xr_7','xr_10']]

#%% Predictors (Features)
FWDS = pd.read_csv(f'data/{period}/fwds_{period}.csv', index_col='Time')            # Forward Rates
MACV = pd.read_csv(f'data/{period}/MacroFactors_{period}.csv', index_col='Time')    # Ludvigson and Ng (2009), Macrovariables
LSC = pd.read_csv(f'data/{period}/lsc_{period}.csv', index_col='Time')              # Level, Slope and Curvature
# YLV = pd.read_csv(f'data/{period}/yl_{period}.csv', index_col='Time')               # yield-level
RVOL = pd.read_csv(f'data/{period}/real_vol_{period}.csv', index_col='Time')        # Realised Volatility 10yr
#IVOL = pd.read_csv(f'data/{period}/imp_vol_{period}.csv', index_col='Time')        # Implied Volatility (Work for 1990~)
CP = pd.read_csv(f'data/{period}/cp_{period}.csv', index_col='Time')                # Cochrane-Piazessi Factor
# YALL = pd.read_csv(f'data/{period}/yl_all_{period}.csv', index_col='Time')          # 12 m, 24 m, ... 120 m
# YMAX = pd.read_csv(f'data/{period}/yl_max_{period}.csv', index_col='Time')          # 1m ... 120 m

F6 = MACV[['F1','F2','F3','F4','F8','F1^3']]
SL = LSC[['slope']]
LV = LSC[['level']]
CU = LSC[['curvature']]


AGG = pd.concat([CP, FWDS], axis=1)

In [12]:
# ────────────────────────────
# Random-Forest
# ────────────────────────────
param_grid_rf = {
    "model__estimator__n_estimators":      [300],
    "model__estimator__max_depth":         [2, 8],
    "model__estimator__min_samples_split": [2, 4],
    "model__estimator__min_samples_leaf":  [1, 2, 4],
    "model__estimator__max_features":      [0.25, 0.5,1],
}

# ────────────────────────────
# Extra-Trees
# ────────────────────────────
param_grid_et = {
    "model__estimator__n_estimators":      [300],
    "model__estimator__max_depth":         [2, 8],
    "model__estimator__min_samples_split": [2, 4],
    "model__estimator__min_samples_leaf":  [1, 2, 4],
    "model__estimator__max_features":      [0.25, 0.5, 1],
}

# ────────────────────────────
# XGBoost  (XGBRegressor 사용 가정)
# ────────────────────────────
param_grid_xgb = {
    "model__estimator__n_estimators":  [300],
    "model__estimator__max_depth":     [2, 4],
    "model__estimator__learning_rate": [0.01],
    "model__estimator__subsample":     [0.7, 0.5],
    "model__estimator__reg_lambda":    [0.1, 1.0],
}

param_grid_lasso = {'reg__alpha': [0.001, 1.0]}    
param_grid_ridge      = {'reg__alpha': [0.001, 1]}
param_grid_elasticnet = {
    'reg__alpha':   [0.01, 0.1, 1, 10],
    'reg__l1_ratio':[0.1, 0.3, 0.5],
}

### Helper

In [ ]:
# ────────────────────────────
# Custom Weights for Portfolio
# ────────────────────────────
w_custom = pd.Series(
    {
        "xr_2" : 2,   # 2Y 듀레이션
        "xr_5" : 5,   # 5Y 듀레이션
        "xr_7" : 7,   # 7Y 듀레이션
        "xr_10": 10,   # 10Y 듀레이션
    },
    name="w"
)

cols_dw = w_custom.index.tolist()
# 합이 1이 되도록 정규화
w_custom = w_custom / w_custom.sum()


# ────────────────────────────
bs = "197108" # burn-in-start
be = "198009" # burn-in-end
p = ["199009", "199009"] #period

### OLS, LASSO, ELASTICNET

In [5]:
ols_1 = Machine(SL, y, 'OLS' , burn_in_start=bs, burn_in_end=be, period=p)
ols_1.training()
ols_2 = Machine(CP,y, 'OLS', burn_in_start=bs, burn_in_end=be, period=p)
ols_2.training()
ols_3 = Machine(F6, y, 'OLS', burn_in_start=bs, burn_in_end=be, period=p)
ols_3.training()

print(ols_1.R2OOS())
print('ew :', ols_1.r2_oos_portfolio())
print('dw :', ols_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ols_2.R2OOS())
print('ew :', ols_2.r2_oos_portfolio())
print('dw :', ols_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ols_3.R2OOS())
print('ew :', ols_3.r2_oos_portfolio())
print('dw :', ols_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

OLS rolling: 100%|██████████| 121/121 [00:03<00:00, 38.58it/s]


xr_2    -0.158961
xr_5    -0.116176
xr_7    -0.103879
xr_10   -0.086677
dtype: float64
ew : -0.41571736803014514
dw : -0.3850221763142885
xr_2     0.200020
xr_5     0.350854
xr_7     0.409726
xr_10    0.429796
dtype: float64
ew : 0.2280802095595056
dw : 0.25900132266376397
xr_2     0.131963
xr_5     0.016148
xr_7    -0.006683
xr_10   -0.063270
dtype: float64
ew : -0.30381352458645927
dw : -0.30407193204084804


In [11]:
ridge_1 = Machine(SL, y, 'Penalized', option='ridge', params_grid=param_grid_ridge, burn_in_start=bs, burn_in_end=be, period=p)
ridge_1.training()
ridge_2 = Machine(CP, y, 'Penalized', option='ridge', params_grid=param_grid_ridge, burn_in_start=bs, burn_in_end=be, period=p)
ridge_2.training()
ridge_3 = Machine(F6, y, 'Penalized', option='ridge', params_grid=param_grid_ridge, burn_in_start=bs, burn_in_end=be, period=p)
ridge_3.training()


print(ridge_1.R2OOS())
print('ew :', ridge_1.r2_oos_portfolio())
print('dw :', ridge_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ridge_2.R2OOS())
print('ew :', ridge_2.r2_oos_portfolio())
print('dw :', ridge_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ridge_3.R2OOS())
print('ew :', ridge_3.r2_oos_portfolio())
print('dw :', ridge_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 121/121 [00:02<00:00, 54.17it/s]


xr_2    -0.158618
xr_5    -0.115986
xr_7    -0.103716
xr_10   -0.086568
dtype: float64
ew : -0.41551959624014323
dw : -0.3848483048070619
xr_2     0.199209
xr_5     0.349482
xr_7     0.408096
xr_10    0.428111
dtype: float64
ew : 0.2260682703260244
dw : 0.2569503082818123
xr_2     0.129903
xr_5     0.015422
xr_7    -0.007117
xr_10   -0.063203
dtype: float64
ew : -0.3043705825884151
dw : -0.3043826012219346


In [6]:
lasso_1 = Machine(SL, y, 'Penalized', option='lasso', params_grid=param_grid_lasso, burn_in_start=bs, burn_in_end=be, period=p)
lasso_1.training()
lasso_2 = Machine(CP, y, 'Penalized', option='lasso', params_grid=param_grid_lasso, burn_in_start=bs, burn_in_end=be, period=p)
lasso_2.training()
lasso_3 = Machine(F6, y, 'Penalized', option='lasso', params_grid=param_grid_lasso, burn_in_start=bs, burn_in_end=be, period=p)
lasso_3.training()


print(lasso_1.R2OOS())
print('ew :', lasso_1.r2_oos_portfolio())
print('dw :', lasso_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(lasso_2.R2OOS())
print('ew :', lasso_2.r2_oos_portfolio())
print('dw :', lasso_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(lasso_3.R2OOS())
print('ew :', lasso_3.r2_oos_portfolio())
print('dw :', lasso_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 121/121 [00:05<00:00, 23.58it/s]


xr_2    -0.115269
xr_5    -0.119344
xr_7    -0.125970
xr_10   -0.108778
dtype: float64
ew : -0.4353839782680553
dw : -0.4092279675171291
xr_2     0.102771
xr_5     0.286915
xr_7     0.367042
xr_10    0.400906
dtype: float64
ew : 0.17076392645819138
dw : 0.21020914770331667
xr_2    -0.115200
xr_5    -0.066707
xr_7    -0.053323
xr_10   -0.060733
dtype: float64
ew : -0.3614514381711533
dw : -0.3376030652615776


In [14]:
elasticnet_1 = Machine(SL, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet, burn_in_start=bs, burn_in_end=be, period=p)
elasticnet_1.training()

elasticnet_2 = Machine(CP, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet, burn_in_start=bs, burn_in_end=be, period=p)
elasticnet_2.training()

elasticnet_3 = Machine(F6, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet, burn_in_start=bs, burn_in_end=be, period=p)
elasticnet_3.training()


print(elasticnet_1.R2OOS())
print('ew :', elasticnet_1.r2_oos_portfolio())
print('dw :', elasticnet_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(elasticnet_2.R2OOS())
print('ew :', elasticnet_2.r2_oos_portfolio())
print('dw :', elasticnet_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(elasticnet_3.R2OOS())
print('ew :', elasticnet_3.r2_oos_portfolio())
print('dw :', elasticnet_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 121/121 [00:03<00:00, 31.57it/s]


xr_2    -0.119808
xr_5    -0.111691
xr_7    -0.107971
xr_10   -0.097341
dtype: float64
ew : -0.4190304742965012
dw : -0.39277647581880504
xr_2     0.113601
xr_5     0.250681
xr_7     0.308855
xr_10    0.339233
dtype: float64
ew : 0.10356794382127033
dw : 0.1384303579607522
xr_2    -0.099949
xr_5    -0.075463
xr_7    -0.071361
xr_10   -0.069186
dtype: float64
ew : -0.3762447561384574
dw : -0.352176868684263


### Tree

In [8]:
rf_1 = Machine(SL, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf, burn_in_start=bs, burn_in_end=be, period=p)
rf_1.training()
rf_2 = Machine(CP, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf, burn_in_start=bs, burn_in_end=be, period=p)
rf_2.training()
rf_3 = Machine(F6, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf, burn_in_start=bs, burn_in_end=be, period=p)
rf_3.training()

print(rf_1.R2OOS())
print('ew :', rf_1.r2_oos_portfolio())
print('dw :', rf_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(rf_2.R2OOS())
print('ew :', rf_2.r2_oos_portfolio())
print('dw :', rf_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(rf_3.R2OOS())
print('ew :', rf_3.r2_oos_portfolio())
print('dw :', rf_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Tree rolling:   0%|          | 0/121 [00:00<?, ?it/s]

Tree rolling: 100%|██████████| 121/121 [39:53<00:00, 19.79s/it]


xr_2    -0.201347
xr_5    -0.205012
xr_7    -0.206031
xr_10   -0.198196
dtype: float64
ew : -0.5435836440223256
dw : -0.5174042371163725
xr_2     0.087766
xr_5     0.264077
xr_7     0.316162
xr_10    0.335771
dtype: float64
ew : 0.11065546727331244
dw : 0.14224115948520144
xr_2    -0.122676
xr_5    -0.164948
xr_7    -0.165331
xr_10   -0.178941
dtype: float64
ew : -0.49846906998514395
dw : -0.4797381627010924


In [9]:
et_1 = Machine(SL, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et, burn_in_start=bs, burn_in_end=be, period=p)
et_1.training()
et_2 = Machine(CP, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et, burn_in_start=bs, burn_in_end=be, period=p)
et_2.training()
et_3 = Machine(F6, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et, burn_in_start=bs, burn_in_end=be, period=p)
et_3.training()


print(et_1.R2OOS())
print('ew :', et_1.r2_oos_portfolio()) 
print('dw :', et_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(et_2.R2OOS())
print('ew :', et_2.r2_oos_portfolio())
print('dw :', et_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(et_3.R2OOS())
print('ew :', et_3.r2_oos_portfolio())
print('dw :', et_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


Tree rolling: 100%|██████████| 121/121 [25:47<00:00, 12.79s/it]


xr_2    -0.163622
xr_5    -0.137096
xr_7    -0.123843
xr_10   -0.103781
dtype: float64
ew : -0.4392612728112901
dw : -0.4082498488035098
xr_2     0.066703
xr_5     0.162794
xr_7     0.199502
xr_10    0.219397
dtype: float64
ew : -0.029442169616374914
dw : -0.0015884337688232275
xr_2    -0.117862
xr_5    -0.102298
xr_7    -0.095463
xr_10   -0.095555
dtype: float64
ew : -0.4088743909896808
dw : -0.3846656460451554


In [13]:
xgb_1 = Machine(SL, y, 'Tree', option='XGB', params_grid=param_grid_xgb, burn_in_start=bs, burn_in_end=be, period=p)
xgb_1.training()
xgb_2 = Machine(CP, y, 'Tree', option='XGB', params_grid=param_grid_xgb, burn_in_start=bs, burn_in_end=be, period=p)
xgb_2.training()
xgb_3 = Machine(F6, y, 'Tree', option='XGB', params_grid=param_grid_xgb, burn_in_start=bs, burn_in_end=be, period=p)
xgb_3.training()

print(xgb_1.R2OOS())
print('ew :', xgb_1.r2_oos_portfolio())
print('dw :', xgb_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(xgb_2.R2OOS())
print('ew :', xgb_2.r2_oos_portfolio())
print('dw :', xgb_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(xgb_3.R2OOS())
print('ew :', xgb_3.r2_oos_portfolio())
print('dw :', xgb_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Tree rolling: 100%|██████████| 121/121 [01:42<00:00,  1.18it/s]


xr_2    -0.176964
xr_5    -0.186073
xr_7    -0.187343
xr_10   -0.164957
dtype: float64
ew : -0.5116631799784046
dw : -0.4834552668241541
xr_2     0.068261
xr_5     0.213247
xr_7     0.267132
xr_10    0.286898
dtype: float64
ew : 0.04772089588124995
dw : 0.07973668586466864
xr_2    -0.138475
xr_5    -0.204236
xr_7    -0.216286
xr_10   -0.243562
dtype: float64
ew : -0.5639841520413531
dw : -0.5502500453830896


### DNN : FWDs ONly

In [ ]:
param_grid = {
    "dnn__module__hidden": [(3,)],    
    "dnn__module__dropout": [0.2],
    "dnn__lr": [0.01],
    "dnn__optimizer__weight_decay": [0.0005],
}

dnn_1 = Machine(
    FWDS, y,
    model_type="DNN",                  
    params_grid=param_grid,
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    forecast_horizon=12,
    random_state=15
)
dnn_1.training()
print(dnn_1.R2OOS())
print('ew :', dnn_1.r2_oos_portfolio())
print('dw :', dnn_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


### DNN : FWDS + MACRO

In [ ]:
param_grid = {
    "dnn__module__hidden": [(3,3)],    
    "dnn__module__dropout": [0.2],
    "dnn__lr": [0.01],
    "dnn__optimizer__weight_decay": [0.0005],
}

dnn_2 = Machine(
    FWDS, y,
    model_type="DNN",                  
    params_grid=param_grid,
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    forecast_horizon=12,
    random_state=15
)
dnn_2.training()
print(dnn_2.R2OOS())
print('ew :', dnn_2.r2_oos_portfolio())
print('dw :', dnn_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


In [ ]:
AGG = pd.concat([MACV, CP], axis=1)

# ── 예시: dual-branch MLP 하이퍼파라미터 그리드 ─────────────────────────
param_grid = {
    # ── Branch-1 Macro
    "dnn__module__hidden1": [(32,)],    # 얕은 vs. 두 층
    "dnn__module__drop1":   [0.2],       # 드롭아웃 비율

    # ── Branch-2 FWDS : CP
    "dnn__module__hidden2": [(1)],
    "dnn__module__drop2":   [0.2],

    # ── 공통 optimizer & regularization ────────────────────────────────
    "dnn__optimizer__lr":           [0.001],
    "dnn__optimizer__weight_decay": [0.0005]
}

grp1_cols = [c for c in AGG.columns if c.startswith("F")]
grp2_cols = [c for c in AGG.columns if c not in grp1_cols]

dnn_d1 = Machine(
    AGG, y,
    model_type="DNN_DUAL",
    option={"grp1": grp1_cols, "grp2": grp2_cols},
    params_grid=param_grid,
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    forecast_horizon=12
)
dnn_d1.training()
print(dnn_d1.R2OOS())

In [ ]:
AGG2 = pd.concat([MACV, FWDS], axis=1)

# ── 예시: dual-branch MLP 하이퍼파라미터 그리드 ─────────────────────────
param_grid = {
    # ── Branch-1 Macro
    "dnn__module__hidden1": [(32,)],    # 얕은 vs. 두 층
    "dnn__module__drop1":   [0.2],       # 드롭아웃 비율

    # ── Branch-2 FWDS : CP
    "dnn__module__hidden2": [(3,3)],
    "dnn__module__drop2":   [0.2],

    # ── 공통 optimizer & regularization ────────────────────────────────
    "dnn__optimizer__lr":           [0.001],
    "dnn__optimizer__weight_decay": [0.0005]
}

grp1_cols = [c for c in AGG.columns if c.startswith("F")]
grp2_cols = [c for c in AGG.columns if c not in grp1_cols]

dnn_d2 = Machine(
    AGG2, y,
    model_type="DNN_DUAL",
    option={"grp1": grp1_cols, "grp2": grp2_cols},
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    params_grid=param_grid,
    forecast_horizon=12
)
dnn_d2.training()
print(dnn_d2.R2OOS())